In [3]:
import numpy as np
# 决策树实现异或XOR，相异为True，相同为False.
# 数据格式[特征1，特征2，标签]
data = np.array([[1, 1, False],[1, 0, True],[0,1, True],[0, 0, False]])
label_index = 2 # 标签所在列


features_num = 2 # 特征的数量
features_possible_value = np.array([[0,1],[0,1]]) #各个特征可能的取值
features_use_info = np.array([False,False]) # 各特征是否已被用来划分数据

def split_set(dataset,feature_index, border_value):

    # 按照指定特征的指定值划分数据集。
    # 划分方法：左子集是指定特征小于等于指定值，右子集是指定特征大于指定值
    left_set_index  = dataset[:,feature_index]<=border_value
    right_set_index = dataset[:,feature_index]>border_value

    return dataset[left_set_index], dataset[right_set_index]

# 计算信息熵，p是当前这个类别占整个集合的比例
def entropy(p):
    if p==0:
        return 0
    else:
        return -p * np.log2(p)

#计算集合的熵
def entropy_dataset(dataset):
    if dataset.shape[0]==0:
        return 0
    # 先计算两种类别的数据的占比
    p_true = (dataset[:,label_index]==True).shape[0]/dataset.shape[0]
    p_false = 1.0 - p_true #（由于只有两个分类所以可以这么算）
    return entropy(p_true) + entropy(p_false)


def make_tree(count,root_data):
    min_entropy_recorder = {}
    #count: 序号，记录是第几次划分
    min_entropy_recorder[count]={'entropy':np.inf}# 记录各种划分方法下的最小的熵和对应划分方法
    tmp_data = root_data

    # 如果数据已经都是一个类了那就不用再划分了
    if np.alltrue(tmp_data[:,label_index]==True):
        return 0,{} #已经都是一个类就熵是0，不用做任何划分动作
    
    #如果特征用完了,只用返回熵，不用做任何动作
    if np.alltrue(features_use_info) or count>features_num:
        return entropy_dataset(root_data),{}

    # 1. 遍历剩下没有使用的特征
    for tmp_feature in np.nditer(np.where(features_use_info==False)):
        # 2. 遍历当前特征各种取值情况
        tmp_values = features_possible_value[tmp_feature]
        features_use_info[tmp_feature]=True
        for v in np.nditer(tmp_values):

            # 3. 按照当前特征的选定值将现在的集合划分为两个子集
            result = split_set(tmp_data,tmp_feature,v)
            print(len(result))
            left_set,right_set  = split_set(tmp_data,tmp_feature,v)
            #递归的划分新的子集
            entropy_left, left_recorder = make_tree(count+1,left_set)
            entropy_right, right_recorder = make_tree(count+1,right_set)
            select_child = ''
            # 哪个子集划分的熵最小就选哪个子集作为划分方案
            if entropy_left<entropy_right:
                select_child = 'left'
                min_entropy_recorder.update(left_set)
            else:
                select_child = 'right'
                min_entropy_recorder.update(right_set)
            # 计算左子集和右子集占总集合的比例
            left_rate = left_set.shape[0]/tmp_data.shape[0]
            right_rate = right_set.shape[0]/tmp_data.shape[0]
            entropy_all = left_rate*entropy_left + right_rate*entropy_left
            
            # 如果当前划分方法是目前最好，那就记录当前的划分方法和熵
            if min_entropy_recorder[count]['entropy']  > entropy_all:
                min_entropy_recorder[count]['entropy'] = entropy_all
                min_entropy_recorder[count]['feature'] = tmp_feature
                min_entropy_recorder[count]['border']  = v
                min_entropy_recorder[count]['select_child']  = select_child
                pass
            pass
        features_use_info[tmp_feature]=False #回溯
        pass #end循环
    return min_entropy_recorder[count]['entropy'], min_entropy_recorder
    pass # end函数


print(make_tree(0,data))

2
2


ValueError: dictionary update sequence element #0 has length 3; 2 is required